In [ ]:
import pandas as pd
import numpy as np
import datetime

#download data, organize into arrays

df = pd.read_csv("mly2275.csv", comment='#')
#print(df.head())

y1=df.iloc[:, 0]
y2=df.iloc[:, 1]

#making y an array of dates 
y=[]
for i in range(np.size(y1, 0)):
    y.append(datetime.date(y1[i], y2[i], 1))
y=np.array(y)

X1=df.iloc[:, 2]
X2=df.iloc[:, 3]
X3=df.iloc[:, 4]
X4=df.iloc[:, 5]
X5=df.iloc[:, 6]
X6=df.iloc[:, 7]
X7=df.iloc[:, 8] 
X8=df.iloc[:, 9]
X9=df.iloc[:, 10]
X10=df.iloc[:, 11]

print(df.dtypes) #some of our columns came out as 'object' (string) --> we'll have to fix that
#                --> where the data's missing, use average of 2 surrounding points It's not perfect, but should be sufficient

for i in range(np.size(X7, 0)):
    if X7[i]=='None':
        X7[i]=(float(X7[i-1])+float(X7[i+1]))/2
X7=X7.astype(float)
for i in range(np.size(X8, 0)):
    if X8[i]=='None':
        X8[i]=(float(X8[i-1])+float(X8[i+1]))/2
X8=X8.astype(float)
for i in range(np.size(X9, 0)):
    if X9[i]=='None':
        X9[i]=(float(X9[i-1])+float(X9[i+1]))/2
X9=X9.astype(float)
for i in range(np.size(X10, 0)):
    if X10[i]=='None':
        X10[i]=None
X10=X10.astype(float)

#we're not including hours of sun in our data, because it's missing for the past decade
X=np.column_stack((X1, X2, X3, X4, X5, X6, X7, X8, X9))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget


f=plt.figure(figsize=(10,9))
f.add_subplot(4, 1, 1)
plt.plot(y[612:None], X1[612:None], color='orange', label='mean temp')
plt.plot(y[612:None], X2[612:None], color='red', label='max temp')
plt.plot(y[612:None], X3[612:None], color='blue', label='min temp')
plt.legend()
f.add_subplot(4, 1, 2)
plt.plot(y[612:None], X4[612:None], color='red', label='mean max temp')
plt.plot(y[612:None], X5[612:None], color='blue', label='mean min temp')
plt.legend()
f.add_subplot(4, 1, 3)
plt.plot(y[612:None], X6[612:None], color='lightsteelblue', label='rain')
plt.plot(y[612:None], X10[612:None], color='gold', label='sun') #note that sun data disappears around 2010 - 2015
plt.legend()
f.add_subplot(4, 1, 4)
plt.plot(y[612:None], X8[612:None], color='plum', label='wind')
plt.plot(y[612:None], X9[612:None], color='purple', label='highest gust')
plt.legend()
plt.show()

In [ ]:
#normalize the data
X1 = np.divide((X1-X1.mean()), np.std(X1))
X2 = np.divide((X2-X2.mean()), np.std(X2))
X3 = np.divide((X3-X3.mean()), np.std(X3))
X4 = np.divide((X4-X4.mean()), np.std(X4))
X5 = np.divide((X5-X5.mean()), np.std(X5))
X6 = np.divide((X6-X6.mean()), np.std(X6))
X7 = np.divide((X7-X7.mean()), np.std(X7))
X8 = np.divide((X8-X8.mean()), np.std(X8))
X9 = np.divide((X9-X9.mean()), np.std(X9))
X10 = np.divide((X10-X10.mean()), np.std(X10))

#visualise normalised data

#timeline - each variable on its own

f=plt.figure(figsize=(10,9))
f.add_subplot(10, 1, 1)
plt.plot(y, X1, color='orange', label='mean temp')
f.add_subplot(10, 1, 2)
plt.plot(y, X2, color='red', label='max temp')
f.add_subplot(10, 1, 3)
plt.plot(y, X3, color='blue', label='min temp')
f.add_subplot(10, 1, 4)
plt.plot(y, X4, color='red',linestyle='dashed', label='mean max temp')
f.add_subplot(10, 1, 5)
plt.plot(y, X5, color='blue',linestyle='dashed', label='mean min temp')
f.add_subplot(10, 1, 6)
plt.plot(y, X6, color='lightsteelblue', label='rain')
f.add_subplot(10, 1, 7)
plt.plot(y, X7, color='green', label='grass mean temp')
f.add_subplot(10, 1, 8)
plt.plot(y, X8, color='plum', label='wind')
f.add_subplot(10, 1, 9)
plt.plot(y, X9, color='purple', label='highest gust')
f.add_subplot(10, 1, 10)
plt.plot(y, X10, color='gold', label='sun') #note that sun data disappears around 2010 - 2015
f.legend()
plt.show()

#last 30 yrs - comparing variables that seem related --> slice [612:None]
f=plt.figure(figsize=(10,9))
f.add_subplot(4, 1, 1)
plt.plot(y[612:None], X1[612:None], color='orange', label='mean temp')
plt.plot(y[612:None], X2[612:None], color='red', label='max temp')
plt.plot(y[612:None], X3[612:None], color='blue', label='min temp')
plt.legend()
f.add_subplot(4, 1, 2)
plt.plot(y[612:None], X4[612:None], color='red', label='mean max temp')
plt.plot(y[612:None], X5[612:None], color='blue', label='mean min temp')
plt.legend()
f.add_subplot(4, 1, 3)
plt.plot(y[612:None], X6[612:None], color='lightsteelblue', label='rain')
plt.plot(y[612:None], X10[612:None], color='gold', label='sun') #note that sun data disappears around 2010 - 2015
plt.legend()
f.add_subplot(4, 1, 4)
plt.plot(y[612:None], X8[612:None], color='plum', label='wind')
plt.plot(y[612:None], X9[612:None], color='purple', label='highest gust')
plt.legend()
plt.show()

In [ ]:
#Ridge regression
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#transform y datetime into a numerical value for the model
y1=[]; y2=[]
for z in range(np.size(y)):
    y1.append(y[z].year)
    y2.append(y[z].month)
y1=np.array(y1);y2=np.array(y2)
y=np.column_stack((y1, y2))


C_range=[1, 5, 10, 20, 50, 75, 100, 150, 200]
MSE=[]
std_error=[]

for C in C_range:
    ridge= Ridge(alpha=1/(2*C))
    kf = KFold(n_splits=5)
    temp=[]
    for train, test in kf.split(X):
        ridge.fit(X[train], y[train])
        ypred = ridge.predict(X[test])
        temp.append(mean_squared_error(y[test],ypred))
    temp=np.array(temp)
    MSE.append(temp.mean())
    std_error.append(temp.std())
MSE=np.array(MSE)
std_error=np.array(std_error)
fig = plt.figure()
plt.errorbar(C_range, MSE, yerr=std_error, linewidth=3)
plt.xlabel('C')
plt.ylabel('Mean square error')
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
C = [1, 5, 10, 25, 100]
m =[]
s = []
for c in C:
    lasso = Lasso(alpha=1/c)
    k = KFold(n_splits=5)
    t = []
    for tr, te in kf.split(X):
        lasso.fit(X[tr], y[tr])   
        yp = lasso.predict(X[te])
        t.append(y[te], ypred)
    t = np.array(t)
    m.append(t.mean())
    s.append(t.std())
plt.errorbar(C, m, yerr=s, linewidth=3)
plt.xlabel('C values')
plt.ylabel('Mean Squared Error')
plt.show()
    
    
   
    